# <center>Agent DQN doszkalany ewolucyjnie</center>
<center><img src="videos\naive_0.gif" width="460" height="280" /></center>

*<center>Wykonali</center>*
*<center>Jakub Gros, Sebastian Kucharzyk</center>*


# <center>Część teoretyczna</center>


## Cel i wstępna analiza problemu
<p style='text-align: justify;'>
Jako cel obraliśmy sobie stworzenie maszynowego odpowiednika człowieka grającego w grę 
komputerową(tzw. agenta), który ucząc się na własnych błędach, będzie osiągał w niej coraz to lepsze 
rezultaty. Do tego celu nadaje się m.in. paradygmat procesu uczenia nazywany uczeniem ze wzmocnieniem,
polegający na umieszczeniu uczącego się agenta w nieznanym mu dotąd środowisku i pozwoleniu na 
wykonywanie dowolnych interakcji z otoczeniem. Agent w trakcie eksploracji doświadcza wielu różnych 
sytuacji zwanych stanami. Znajdując się w danym stanie wybiera, którą z obecnie możliwych akcji 
wykona, za co  finalnie otrzymuje pewną nagrodę lub karę. Wszystkie informacje na temat ruchów i 
nagród przechowuje w pamięci, aby nastepnie na ich podstawie móc oceniać, jaka akcja da mu 
największą nagrodę.
</p>


<p style='text-align: justify;'>
Pozornie wydawałoby się, że rozwiązanie to nadaje się niemal idealnie do sytuacji 
wymagających, aby agent sam zebrał dane, na których następnie będzie się uczył, ale niestety posiada ono 
jedną znaczącą wadę - wraz z upływem czasu ilość danych w pamięci agenta rośnie, co z kolei powoduje 
spowolnienie podejmowania decyzji, a finalnie może nawet doprowadzić do zapełnienia pamięci 
fizycznej urządzenia.
</p>


<p style='text-align: justify;'>
Aby uniknąć problemów związanych z nadmiarem danych, zdecydowaliśmny się na wykorzystanie innego paradygmatu, 
który pozwala zgeneralizować zbiór danych zamiast nieskończenie go poszerzać i przeszukiwać. Dokładniej 
rzecz ujmując, wybór padł na zastosowanie głębokiego Q-learningu, czyli połączenia uczenia 
ze wzmocnieniem z uczeniem głębokim. Polega on na zastąpieniu nieograniczonej pamięci agenta siecią neuronową 
wspomaganą relatywnie małym buforem pamiętającym N ostatnich wyborów. Ponadto po każdej akcji podjętej przez agenta
z buforu wybierany jest mały fragment danych, na których następnie sieć jest dotrenowywana.
Wykonuje się to za pomocą Q-wartości, czyli pewnej funkcji  <b>Q(S, A)</b> szacującej jak bardzo opłaca się 
podjąć akcję A będąc w stanie S. Cały powyższy proces prowadzi do stworzenia modelu Deep Q-Network(DQN).
</p>


<p style='text-align: justify;'>
Tutaj zakres projektu mógłby się zakończyć, ale nie chcieliśmy się ograniczać jedynie do głębokiego Q-learningu, 
więc postanowiliśmy znaleźć zastosowanie dla innego zagadnienia z zakresu stucznej inteligencji - algorytmów 
ewolucyjnych. Patrząc na sposób działania modelu DQN można zauważyć trzy obszary, które potencjalnie nadają się 
do ulepszenia ewolucyjnego:
</p>

1. Kształt sieci neuronowej
2. Wagi połączeń międzyneuronowych
3. Dane przechowywane w pamięci agentów


<p style='text-align: justify;'>
Po dokładnym przeanalizowaniu powyższych możliwości zdecydowaliśmy się odrzucić opcje numer 1 oraz 3. 
O wyeliminowaniu manipulacji kształtem sieci neuronowej zadecydował fakt, że wolimy skupić się na
stworzeniu jednego wariantu sieci i jego dogłębnej analizie niż na pobieżnym tworzeniu wielu modeli tylko 
po to, żeby różniły się budową. Natomiast jeśli chodzi o opcję trzecią, to jej odrzucenie wynika z 
natury DQN, której agent przechowuje w pamięci jedynie mały  fragment z całego zbioru danych, 
na którym był uczony. W takim wypadku używanie go jako podstawy do dalszego ulepszania najprawdopodobniej nie 
prowadziłoby do uzyskania wymiernych korzyści. W ten sposób stwierdziliśmy, że najciekawszym i (oby) 
najefektywniejszym podejściem będzie zastosowanie algorytmu genetycznego operującego na wagach połączeń 
międzyneuronowych.
</p>


## Etapy działania
Podsumowując przeprowadzoną powyżej analizę, zdecydowaliśmy się podzielić realizację projektu na następujące 
fazy:
1. Implementacja i gromadzenie rezultatów naiwnego agenta, który podejmuje decyzje w sposób losowy. Posłuży on 
jako wyznacznik do porównywania innych modeli.
2. Implementacja i gromadzenie rezultatów agenta wykorzystującego prostą sieć neuronową.
3. Modyfikacja implementacji agenta oraz parametrów sieci neuronowej w celu poprawy osiąganych rezultatów.
4. Wytrenowanie początkowej populacji agentów i zastosowanie algorytmu genetycznego do stworzenia osobnika z 
jak najlepiej dostosowanymi wagami międzyneuronowymi.
5. Porównanie osiągniętych rezultatów i wyciągnięcie wniosków z doświadczenia.


## Ocena jakości modelu
<p style='text-align: justify;'>
Aby rzetelnie ocenić jakość modelu, należy najpierw upewnić się, że wyniki przez niego osiągane są stabilne oraz 
wiarygodne. Mianem stabilnych określamy je wtedy, gdy w trakcie rozgrywania kolejnych gier przyjmują one zbliżone 
do siebie wartości. Aby osiągnąć taki rezultat, wymagane jest trenowanie modelu na odpowiedniej 
(nie za małej) liczbie gier. Metodą prób i błędów określiliśmy, że dla różnych modeli w naszym środowisku treningowym
liczba ta oscyluje przeważnie w granicach 200.
</p>


<p style='text-align: justify;'>
Z kolei jeśli chodzi o wiarygodność, to uzyskiwane przez dany model rezultaty uznajemy za wiarygodne wtedy, gdy 
nie są one wypaczone przez losowe warunki początkowe. Istnieją dwa różne podejścia pomagające w zapewnieniu 
wiarygodności wyników:
</p>

1. Trenowanie każdego modeli w identycznych warunków początkowych
2. Zastosowanie pewnej funkcji oceny, np. wyciągnięcia średniej z kilku różnych prób wytrenowania tego samego modelu.

<p style='text-align: justify;'>
Ze względu na to, że ideą głębokiego Q-learningu jest eksploracja środowiska w sposób losowy, nasz wybór musiał 
paść na opcję numer 2.
</p>


<p style='text-align: justify;'>
Zważając na powyższe rozważania zdecydowaliśmy, że każdy z naszych modeli będzie trenowany dokładnie cztery 
razy po dwieście gier. Z rezultatów osiąganych we wszystkich czterech podejściach wyciągniemy średnią, a 
następnie porównamy ją z innymi modelami.



## Zastosowane technologie
<p style='text-align: justify;'>
Kod projektu napisaliśmy w języku Python w wersji 3.6. Sieć neuronowa oraz wszelkie struktury i metody
z nią  związane pochodzą z wysokopoziomowego API Kerasa bazującego na bibliotece TensorFlow. Jeśli zaś 
chodzi o samo środowisko do trenowania agenta, wykorzystaliśmy zestaw narzędzi AI Gym, a dokładniej
grę Cart Pole. Ponadto, aby maksymalnie wykorzystać możliwości sprzętowe zdecydowaliśmy, że 
części kodu wymagające wykonywania sekwencyjnego będą uruchamiane z użyciem procesora, a fragmenty 
możliwe do zrównoleglenia obsłuży karta graficzna. 
</p>


## Dygresja na temat podjętych decyzji
<p style='text-align: justify;'>
Pierwszą wersję modelu stworzyliśmy z myślą o uniwersalności. Zasadniczo była ona zdolna do współpracy z 
większością środowisk dostępnych poprzez interfejs AI Gym. Dzięki temu zabiegowi byliśmy w stanie przeprowadzić 
serię prób pozwalających na wyłonienie najlepszego kandydata do dalszego ulepszania modelu. Testy zaczęliśmy od 
najbardziej złożonych środowisk (BipedalWalker-v2 - kombinacje 8 różnych możliwych do podjęcia akcji) poprzez te średnio 
skomplikowane (LunarLander-v2 - możliwe do wykonania 4 akcje) aż po najłatwiejsze (CartPole-v1 - tylko 2 możliwe akcje). 
Niestety okazało się, że nasz sprzęt nie jest w stanie wytrenować sieci neuronowej dla najtrudniejszego 
środowiska w racjonalnych ramach czasowych, więc postanowiliśmy skupić się na grze o średniej truności - Lunar Lander.
</p>


<p style='text-align: justify;'>
Mówiąc konkretniej, samo trenowanie jednego modelu do stopnia, w którym jego wyniki są powtarzalne i pozwalają na 
wyciąganie sensownych wniosków, zajmuje około 10 minut w najprostszym środowisku (CartPole), w drugim 
co do złożoności LunarLander czas ten jest około 5 krotnie większy, natomiast w BipedalWalker jest aż 20 krotnie większy. 
Idąc dalej, po oszacowaniu, że do zakończenia projektu będziemy musieli trenować model w różnych konfiguracjach blisko 
100 razy, najtrudniejszym środowiskiem na jakie mogliśmy sobie pozwolić był Lunar Lander.
</p>


## Opis środowiska LunarLander-v2
<p style='text-align: justify;'>
Lądowisko ma zawsze współrzędne (0,0). Współrzędne to dwie pierwsze liczby w wektorze stanu. Nagroda za 
przebycie drogi od góry ekranu aż do lądowiska i do prędkości 0 to około 100-140 punktów. 
Gra kończy się w momencie, gdy lądownik rozbija się lub ląduje, otrzymując dodatkowe -100 lub +100 punktów. Każda noga,
która styka się z ziemią to dodatkowe +10 punktów. Uruchamianie głównego silinika to -0.3 punkta na klatkę. Skuteczne 
lądowanie to 200 punktów. Lądowanie poza lądowiskiem jest możliwe. Paliwo jest nieskończone, więc agent może najpierw 
nauczyć się latać, a potem wylądować za pierwszym razem. Możliwe do wykonania 4 dyskretne akcje: nie rób nic, uruchom 
lewy silnik, uruchom główny silnik, uruchom prawy silnik przyjmują kolejno wartości 0, 1, 2 oraz 3.
</p>


***


# <center>Część praktyczna</center>
## Preludium
<p style='text-align: justify;'>
W poniższych podrozdziałach zawarliśmy przegląd wszystkich wykonanych przez nas modeli. Przedstawione są one w 
sposób chronologiczny względem ich powstania. Prezentacji podlega zarówno kod źródłowy jak i osiągane wyniki. 
Ponadto pozwoliliśmy sobie pominąć komentowanie oczywistych części kodu, aby nie zaciemniać tego, 
co najważniejsze, czyli rezultatów i wniosków z tworzenia kolejnych modeli. Natomiast jeśli chodzi o fragmenty,
które uznaliśmy za ważne, to są one opisane bezpośrednio w kodzie lub w kolejnym akapicie pod nim.
</p>

***



## <b>Naiwny agent losowy</b>
Pierwszy zaimplementowany przez nas agent podejmuje decyzje o tym, jaką akcę wykonać, w sposób losowy. Stanowi 
swego rodzaju obiekt porównawczy dla późniejszych modeli. Jego implementacja wygląda następująco:


In [ ]:
import random
import gym     # Pełni rolę interfejsu dla środowiska LunarLander

In [ ]:
NUM_OF_AGENTS = 4          # liczba agentów którzy będą wytrenowani od podstaw.
NUM_OF_EPISODES = 100      # liczba gier rozegranych przez  jednego agenta.
FRAMES_PER_EPISODE = 1000  # liczba klatek równa liczbie akcji do podjęcia przez agenta w każdej grze
GAME_ID = "LunarLander-v2"

In [3]:
game = gym.make(GAME_ID)             # Tworzymy instancję określonej wcześniej gry
num_of_actions = game.action_space.n # wyłuskujemy ilość akcji, które agent ma do wyboru
is_done = False
avgs = []
for model in range(NUM_OF_AGENTS): 
    scores = []
    for episode in range(NUM_OF_EPISODES):
        current_state = game.reset()
        total_score = 0
        for frame in range(FRAMES_PER_EPISODE):
            # game.render()
            action = random.randrange(num_of_actions) # wybór losowej akcji z zakresu możliwych
            new_state, gained_reward, is_done, info = game.step(action) 
            total_score += gained_reward
            if is_done:
                break
        scores.append(total_score)             # zapamiętujemy wynik po zakończeniu gry
    avgs.append(sum(scores)/len(scores)) # zapamiętujemy średni wynik osiągniety przez agenta
game.close()

> Do wytrenowania agenta potrzebujemy dwóch metod z interfejsu <b>gym</b> operujących na grze:
1. <b>reset()</b> - inicjalizuje nową grę oraz zwraca stan początkowy agenta
2. <b>step(action)</b> - wykonuje akcję podaną jako argument, zwraca kolejno: nowy stan, zdobytą nagrodę, flagę wskazującą 
czy gra się zakończyła oraz obiekt zawierający informacje dla programisty (debug info).

In [5]:
for i, avg in enumerate(avgs): # Wyświetlamy wyniki poszczególnych agentów
    print("Model {} has avarage: {}".format(i, avg))
print("Overall avg: {}".format(sum(avgs)/len(avgs)))

Model 0 has avarage: -181.27832294252718
Model 1 has avarage: -191.51285271308996
Model 2 has avarage: -182.1684255640586
Model 3 has avarage: -194.19551136274666
Overall avg: -187.2887781456056



### Wnioski
<p style='text-align: justify;'>
Działania agenta były losowe, więc zgodnie z przewidywaniami osiągane przez niego wyniki były słabe. Poniżej można 
zobaczyć krótkie nagranie przedstawiające sposób jego działania:
</p>


<center><img src="videos\naive_0.gif" width="460" height="280" /></center>

***


## Agent DQN v1
<p style='text-align: justify;'>
Następnym krokiem było usprawnienie naiwnego agenta poprzez dodanie prostej sieci neuronowej oraz metod
odpowiedzialnych za jej trenowanie. 
</p>


In [ ]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
import tensorflow as tf # używamy go do określenia, który fragment kodu wykonuje się na GPU

In [ ]:
class SimpleDqnNpc:
    "Klasa implementująca agenta DQN opartego o prostą sieć neuronową"

    def __init__(self, num_of_inputs, num_of_outputs):
        """
        num_of_inputs - długość wektora będącego wejściem dla sieci neuronowej
        num_of_outputs - ilość wyjść z sieci neuronowej
        """

        self._num_of_inputs = num_of_inputs
        self._num_of_outputs = num_of_outputs
        self._exploration_rate = 1.0           # Prawdopodobieństwo podjęcia losowej akcji
        self._exploration_rate_min = 0.1
        self._exploration_rate_decay = 0.997
        self.memory = deque(maxlen=1024)
        self._init_model()

    def _init_model(self):
        """
        Inicjalizuje model sieci neuronowej.
        """

        self._model = Sequential()
        self._model.add(Dense(self._num_of_inputs, input_dim=self._num_of_inputs, activation='linear'))
        self._model.add(Dense(self._num_of_outputs, activation='linear'))
        self._model.compile(optimizer=SGD(), loss='mean_squared_error')

    def act(self, state):
        """Przewiduje i zwraca akcję, którą należy wykonać"""

        if np.random.rand() <= self._exploration_rate:
            return random.randrange(self._num_of_outputs)
        act_values = self._model.predict(state)
        return np.argmax(act_values[0])

    def retain(self, current_state, taken_action, gained_reward, future_state, is_lost):
        """Zapisuje dany przypadek w pamięci agenta"""

        self.memory.append((current_state, taken_action, gained_reward, future_state, is_lost))

    def replay(self, batch_size):
        """
        Doszkala sieć neuronową na losowym fragmencie z jego pamięci
        batch-size - rozmiar fragmentu pamięci
        """

        batch = random.sample(self.memory, batch_size)
        for current_state, taken_action, gained_reward, future_state, is_done in batch:
            next_act_best_profit = gained_reward
            if not is_done:
                future_act_profits = self._model.predict(future_state)
                next_act_best_profit = np.amax(future_act_profits[0])
            current_act_profits = self._model.predict(current_state)
            current_act_profits[0][taken_action] = next_act_best_profit
            with tf.device('/device:GPU:0'):
                self._model.fit(x=current_state, y=current_act_profits, epochs=1, verbose=0)
        if self._exploration_rate > self._exploration_rate_min:
            self._exploration_rate *= self._exploration_rate_decay
        else:
            self._exploration_rate = 0.0

    def load(self, model_path):
        """Wczytuje model z pamięci"""

        self._model.load_weights(model_path)

    def save(self, model_path):
        """Zapisuje modele do pamięci"""

        self._model.save_weights(model_path)

> <p style='text-align: justify;'>
Sieć neuronowa zaimplementowana w metodzie <b>_init_model()</b> została stworzona jako proste odzwierciedlenie posiadanych
przez nas danych, czyli warstwa wejściowa ma ilość neuronów równą rozmiarowi danych wejściowych, a wyjściowa równą 
ilości możliwych do podjęcia akcji. Parametry takie jak funkcja aktywacji czy straty na razie wybraliśmy losowo.
<br/>
Implementacja metody <b>replay(batch_size)</b> jest jedną z najprostszych powszechnie stosowanych w modelach DQN. 
Wybiera małą próbkę danych (aby uniknąć znacznego spowolnienia całego procesu), a następnie dotrenowuje na niej model poprzez
próbę przewidzenia najlepszej decyzji dla następnego stanu i skojarzenia jej z obecnym stanem. Można sobie to
wyobrażać jako coraz dalsze i dalsze spoglądanie w przyszłość. Dzięki zastosowaniu podwójnie zakończonej kolejki
(deque) jako pamięci, zyskujemy duże prawdopodobieństwo na kilkukrotne dotrenowanie modelu na danym przypadku, a jednocześnie
uniknięcie przetrenowania. Dodatkowo po każdym wywołaniu tej metody obniżamy <b>współczynnik eksploracji</b> agenta, 
czyli zmniejszamy szansę na wykonanie losowego ruchu.
</p>

In [ ]:
NUM_OF_AGENTS = 4
NUM_OF_EPISODES = 200
FRAMES_PER_EPISODE = 1000
BATCH_SIZE = 16              # Rozmiar próbki danych do dotrenowywania modelu
GAME_ID = "LunarLander-v2"

In [ ]:
game = gym.make(GAME_ID)
num_of_actions = game.action_space.n
observation_size = game.observation_space.shape[0]
npc = SimpleDqnNpc(observation_size, num_of_actions)
is_done = False
avgs = []

In [ ]:
for model in range(NUM_OF_AGENTS):
    scores = []
    for episode in range(NUM_OF_EPISODES):
        total_score = 0
        current_state = np.reshape(game.reset(), [1, observation_size])
        for frame in range(FRAMES_PER_EPISODE):
            # game.render()
            action = npc.act(current_state)
            new_state, gained_reward, is_done, info = game.step(action)
            new_state = np.reshape(new_state, [1, observation_size])
            npc.retain(current_state, action, gained_reward, new_state, is_done)
            total_score += gained_reward
            current_state = new_state
            if len(npc.memory) > BATCH_SIZE:
                npc.replay(BATCH_SIZE)
            if is_done:
                break
        scores.append(total_score)
    npc.save("simple_dqn_" + str(model) + ".h5")


> <p style='text-align: justify;'>
Zmiana kształtu zmiennych przechowujących stany wynika z implementacji kerasa. Żeby dane zostały poprawnie 
zinterpretowane należy umieścić je w zagnieżdżonych listach.
</p>

In [ ]:
for i, avg in enumerate(avgs):
    print("Model {} has avarage: {}".format(i, avg))
print("Overall avg: {}".format(sum(avgs) / len(avgs)))

Model 0 has avarage: -136.84274049982846
Model 1 has avarage: -142.73024592812233
Model 2 has avarage: -137.12955212352132
Model 3 has avarage: -137.8901023058812
Overall avg: -138.64816021433834



### Wnioski
<p style='text-align: justify;'>
Liczbowe wyniki agenta z prostą siecią neuronową są nieco lepsze od wyników agenta naiwnego. Po analizie wydarzeń oraz 
zapisu wideo można zauważyć, że agent rzeczywiście początkowo skupia się na eksploracji otoczenia
uruchamiając silniki w przypadkowej kolejności. Niemniej jednak wraz z upływem kolejnych prób i zmniejszania się 
współczynnika eksploracji, zaczyna podejmować najlepsze (według niego) decyzje, czyli po prostu opadać w dół. 
Poniekąd jest to racjonalna decyzja, bo właśnie na tym polega Lunar Lander, jednakże agent nie nauczył się zwracać 
uwagi na te mniej oczywiste czynniki wspomagające sukces, takie jak stabilizacja statku czy niezbaczanie z pionowej
trajektorii lotu. Wygląda na to, że model bardzo szybko wpadał w minimum lokalne. Mogło się tak stać, ponieważ po pewnym czasie 
ustaliliśmy wartość współczynnika eksploracji na 0, więc agent nie miał szansy odkryć niczego nowego.
</p>



<p style='text-align: justify;'>
Ponadto na ten efekt złożył się też fakt, że kształt i parametry naszej sieci neuronowej nie są zbyt dobrze dobrane. 
Przede wszystkim liniowa funkcja aktywacji dla obu warstw to marnowanie potencjału wynikającego ze stosowania sieci neuronowej. 
Jeśli dana warstwa neuronów używa liniowej funkcji aktywacji do określenia pobudzenia poszczególnych neuronów, a 
następnie przekazuje je jako impuls do kolejnej warstwy pobudzanej liniowo, to na końcu nie otrzymujemy niczego innego 
niż po prostu pewną funkcję liniową określoną na podstawie wartości wejść pierwszej. To oznacza, że wszystkie znajdujące się
obok siebie warstwy z liniową funkcją aktywacji mogą po prostu zostać zamienione w jedną warstwę. Innym problemem funkcji
liniowej jest to, że w miarę dotrenowywania jej wartości na wyjściu mogą zmierzać do nieskończoności, aż w pewnym
momencie przepełnić stos i być interpretowane jako <i>NaN</i>.


<p style='text-align: justify;'>
Drugim, poza funkcją straty, rzucającym się w oczy parametrem jest funkcja optymalizacji. Użyliśmy algorytmu 
stochastycznego spadku wzdłuż gradientu, który skupia się na określeniu tendencji optymalizowanej funkcji 
biorąc pod uwagę jedynie położenie, w którym znajduje się w danym momencie. Jeśli zastanowimy się dłużej 
nad tym, jaki efekt chcemy osiągnąć w modelu DQN, to dojdziemy do wniosku, że ważne jest dla nas skupianie 
się nie na jednym, a na kilku krokach naprzód. 
</p>


<p style='text-align: justify;'>
Poza tym jeśli spojrzymy na metody wykorzystywane przez  DQN, to też nie są one bez wad. Na przykład wewnątrz
metody replay w trakcie przewidywania wartości dla następnego stanu i wpisywania jej do obecnego, nie skalujemy 
jej w żaden sposób.  To oznacza, że bezgranicznie wierzymy naszemu modelowi, że skutecznie przewiduje ruch, który
należy wykonać (chociaż w cale tak nie jest). Również w tej samej metodzie w trakcie trenowania na stanach, które
nie są stanami kończącymi grę, nie bierzemy pod uwagę zdobytych przez nie nagród.
</p>


<center><img src="videos\v1_0.gif" width="460" height="280" /></center>


## Agent DQN v2
<p style='text-align: justify;'>
Biorąc pod uwagę wnioski, które wyciągnęliśmy z poprzedniego model, zdecydowaliśmy się na wykonanie następujących
czynności:
</p>

1. Dodanie współczynnika niepewności <b>discountrate</b> wobec przewidzianego wyniku oraz uwzględnienie otrzymanej 
nagrody w każym przypadku.
2. Rezygnacja z zerowania wartości współczynnika uczenia po osiągnięciu progu 0.1.
3. Zmiana funkcji aktywacji pierwszej warstwy na <b>relu</b> ze względu na jej charakterystykę - dla wartości mniejszych od 
zera zwraca 0, dzięki czemu istnieje duże prawdopodobieństwo, że część neuronów nie zostanie pobudzona, 
co sprawi, że sieć neuronowa będzie działała szybciej, a w naszym przypadku jest to bardzo duża zaleta. Niemniej jednak
w połączeniu z funkcją liniową na wyjściu, wciąż mógłby pojawić się problem z wartościami na wyjściu zmierzającymi do nieskończoności, więc zdecydowaliśmy się 
dodać ukrytą warstwę z sigmoidalną funkcją aktywacji, która zwraca wartości od 0 do 1, a więc skutecznie przeskaluje wyniki.
4. Zmiana funkcji optymalizacji na <b>ADAM</b>, która usprawnia scholastyczny algorytm gradientu o dodanie wartości pędu, co 
pozwala na unikanie minimów lokalnych. Działa to zupełnie jak w fizyce - jeśli kula toczy się w dół zbocza i na drodze 
napotka kamień, to mimo wszystko będzie w stanie się po nim przetoczyć dzięki nabranemu wcześniej pędowi, więc ominie 
swoje minimum lokalne. 


In [ ]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import tensorflow as tf

In [ ]:
class SimpleDqnNpcV2:
    "Klasa implementująca agenta DQN opartego o prostą sieć neuronową"

    def __init__(self, num_of_inputs, num_of_outputs):
        """
        num_of_inputs - długość wektora będącego wejściem dla sieci neuronowej
        num_of_outputs - ilość wyjść z sieci neuronowej
        """

        self._num_of_inputs = num_of_inputs
        self._num_of_outputs = num_of_outputs
        self._exploration_rate = 1.0
        self._exploration_rate_min = 0.1
        self._exploration_rate_decay = 0.997
        self._discout_rate = 0.95   # wskaźnik "nieufności" wobec predykcji
        self.memory = deque(maxlen=1024)
        self._init_model()

    def _init_model(self):
        """
        Inicjalizuje model sieci neuronowej.
        Wybraliśmy (w naszym mniemaniu) najproszte parametry i kształt.
        """

        self._model = Sequential()
        self._model.add(Dense(self._num_of_inputs, input_dim=self._num_of_inputs, activation='relu'))
        self._model.add(Dense(self._num_of_inputs * 2, activation='sigmoid')) 
        self._model.add(Dense(self._num_of_outputs, activation='linear'))
        self._model.compile(optimizer=Adam(), loss='mean_squared_error')

    def act(self, state):
        """Przewiduje i zwraca akcję, którą należy wykonać"""

        if np.random.rand() <= self._exploration_rate:
            return random.randrange(self._num_of_outputs)
        act_values = self._model.predict(state)
        return np.argmax(act_values[0])

    def retain(self, current_state, taken_action, gained_reward, next_state, is_done):
        """Zapisuje dyn przypadku w pamięci agenta"""

        self.memory.append((current_state, taken_action, gained_reward, next_state, is_done))

    def replay(self, batch_size):
        """
        Doszkala sieć neuronową na losowym fragmencie z jego pamięci
        batch-size - rozmiar fragmentu pamięci
        """

        batch = random.sample(self.memory, batch_size)
        for current_state, taken_action, gained_reward, next_state, is_done in batch:
            next_act_best_profit = gained_reward
            if not is_done:
                future_act_profits = self._model.predict(next_state)
                next_act_best_profit = gained_reward + self._discout_rate * np.amax(future_act_profits[0])
            current_act_profits = self._model.predict(current_state)
            current_act_profits[0][taken_action] = gained_reward + self._discout_rate * next_act_best_profit
            with tf.device('/device:GPU:0'):
                self._model.fit(x=current_state, y=current_act_profits, epochs=1, verbose=0)
        if self._exploration_rate > self._exploration_rate_min:
            self._exploration_rate *= self._exploration_rate_decay

    def load(self, model_path):
        """Wczytuje model z pamięci"""

        self._model.load_weights(model_path)

    def save(self, model_path):
        """Zapisuje modele do pamięci"""

        self._model.save_weights(model_path)

In [ ]:
NUM_OF_AGENTS = 4
NUM_OF_EPISODES = 200
FRAMES_PER_EPISODE = 1000
BATCH_SIZE = 16
GAME_ID = "LunarLander-v2"

In [ ]:
game = gym.make(GAME_ID)
num_of_actions = game.action_space.n
observation_size = game.observation_space.shape[0]
npc = SimpleDqnNpcV2(observation_size, num_of_actions)
is_done = False
avgs = []

In [ ]:
for model in range(NUM_OF_AGENTS):
    scores = []
    for episode in range(NUM_OF_EPISODES):
        score = 0
        current_state = np.reshape(game.reset(), [1, observation_size])
        for frame in range(FRAMES_PER_EPISODE):
            # game.render()
            action = npc.act(current_state)
            new_state, gained_reward, is_done, info = game.step(action)
            new_state = np.reshape(new_state, [1, observation_size])
            npc.retain(current_state, action, gained_reward, new_state, is_done)
            score += gained_reward
            current_state = new_state
            if len(npc.memory) > BATCH_SIZE:
                npc.replay(BATCH_SIZE)
            if is_done:
                break
        scores.append(score)
    npc.save("simple_dqn_" + str(model) + ".h5")
    avgs.append(sum(scores) / len(scores))

In [ ]:
for i, avg in enumerate(avgs):
    print("Model {} has avarage: {}".format(i, avg))
print("Overall avg: {}".format(sum(avgs) / len(avgs)))

Model 0 has avarage: -168.23305719787398
Model 1 has avarage: -145.7944561029682
Model 2 has avarage: -173.22079611413694
Model 3 has avarage: -158.50128312239501
Overall avg: -161.43739813434353



### Wnioski
<p style='text-align: justify;'>
Niestety tym razem wyniki agenta w liczbach prezentują się nieco gorzej. Aczkolwiek duży progres można zauważyć po 
przejrzeniu nagrań wideo. Przez około 70 pierwszych gier agent wyraźnie próbuje utrzymać trajektorię lotu w taki 
sposób, aby kolizja z podłożem miała miejsce dokładnie na lądowisku:
</p>


<center><img src="videos\v2_0.gif" width="460" height="280" /></center>


<p style='text-align: justify;'>
Po rozegraniu około 130 gier agent zaczął wypracowywać technikę bujania się w lewo i w prawo, aby następnie bezpiecznie 
wylądować(z winikami około -30pkt!), co z resztą kilkukrotnie mu się udało (niestety nie na lądowisku):
</p>


<center><img src="videos\v2_1.gif" width="460" height="280" /></center>


<p style='text-align: justify;'>
Gdy agent osiągnął próg blisko 160 gier okazało się, dlaczego wynik tego modelu jest gorszy od poprzednika.
Agent po prostu zaniechał wszelkich prób lądowania i zaczął jedynie utrzymywać się nieruchomo w 
powietrzu:
</p>


<center><img src="videos\v2_2.gif" width="460" height="280" /></center>


Stwierdziliśmy, że może to być efekt przetrenowania i postanowiliśmy przyjrzeć się temu podczas tworzenia kolejnego 
modelu.



## <b>Agent DQN v3</b>
<p style='text-align: justify;'>
Mając na względzie wnioski po poprzednim modelu postanowiliśmy wykonać następujące modyfikacje:
</p>

1. Zwiększenie pamięc agenta, żeby douczał się na danych z szerszego okresu. Dzięki temu unikniemy sytuacji, 
w których agent spędzając zbyt dużo czasu w prawie niezmienionym stanie, będzie wielokrotnie trenował się na bardzo 
podobnych do siebie danych.
2. Poszerzenie dwóch pierwszych warstw sieci neuronowej, aby uzyskać dodatkowe kombinacje i połączenia tworzące 
więcej ścieżek do aktywacji.
3. Dodanie do każdej warstwy funkcji Dropout, która w trakcie trenowania dezaktywuje część neuronów równą
przyjętemu jako argument ułamkowi.

<p style='text-align: justify;'>
Podjęliśmy również próbę zwiększenia rozmiaru próbki doszkalającej (BATCH_SIZE), ale niestety szybko wycofaliśmy się 
 z tego pomysłu, bo po raz kolejny dały o sobie znać ograniczenia sprzętowe. 
</p>


In [ ]:
from keras.layers import Dropout
from keras.optimizers import Adam

In [ ]:
class SimpleDqnNpcV3:
    "Klasa implementująca agenta DQN opartego o prostą sieć neuronową"

    def __init__(self, num_of_inputs, num_of_outputs):
        """
        num_of_inputs - długość wektora będącego wejściem dla sieci neuronowej
        num_of_outputs - ilość wyjść z sieci neuronowej
        """

        self._num_of_inputs = num_of_inputs
        self._num_of_outputs = num_of_outputs
        self._exploration_rate = 1.0
        self._exploration_rate_min = 0.1
        self._exploration_rate_decay = 0.997
        self._discout_rate = 0.95
        self.memory = deque(maxlen=4096)
        self._init_model()

    def _init_model(self):
        """
        Inicjalizuje model sieci neuronowej.
        Wybraliśmy (w naszym mniemaniu) najproszte parametry i kształt.
        """

        self._model = Sequential()
        self._model.add(Dense(5 * self._num_of_inputs, input_dim=self._num_of_inputs, activation='relu'))
        self._model.add(Dropout(0.15))
        self._model.add(Dense(4 * self._num_of_inputs, activation='sigmoid'))
        self._model.add(Dropout(0.15))
        self._model.add(Dense(self._num_of_outputs, activation='linear'))
        self._model.compile(optimizer=Adam(), loss='mean_squared_error')

    def act(self, state):
        """Przewiduje i zwraca akcję, którą należy wykonać"""

        if np.random.rand() <= self._exploration_rate:
            return random.randrange(self._num_of_outputs)
        act_values = self._model.predict(state)
        return np.argmax(act_values[0])

    def retain(self, current_state, taken_action, gained_reward, next_state, is_done):
        """Zapisuje dyn przypadku w pamięci agenta"""

        self.memory.append((current_state, taken_action, gained_reward, next_state, is_done))

    def replay(self, batch_size):
        """
        Doszkala sieć neuronową na losowym fragmencie z jego pamięci
        batch-size - rozmiar fragmentu pamięci
        """

        batch = random.sample(self.memory, batch_size)
        for current_state, taken_action, gained_reward, next_state, is_done in batch:
            next_act_best_profit = gained_reward
            if not is_done:
                future_act_profits = self._model.predict(next_state)
                next_act_best_profit = gained_reward + self._discout_rate * np.amax(future_act_profits[0])
            current_act_profits = self._model.predict(current_state)
            current_act_profits[0][taken_action] = gained_reward + self._discout_rate * next_act_best_profit
            with tf.device('/device:GPU:0'):
                self._model.fit(x=current_state, y=current_act_profits, epochs=1, verbose=0)
        if self._exploration_rate > self._exploration_rate_min:
            self._exploration_rate *= self._exploration_rate_decay

    def load(self, model_path):
        """Wczytuje model z pamięci"""

        self._model.load_weights(model_path)

    def save(self, model_path):
        """Zapisuje modele do pamięci"""

        self._model.save_weights(model_path)

In [ ]:
NUM_OF_AGENTS = 4
NUM_OF_EPISODES = 200
FRAMES_PER_EPISODE = 1000
BATCH_SIZE = 16
GAME_ID = "LunarLander-v2"

In [ ]:
num_of_actions = game.action_space.n
observation_size = game.observation_space.shape[0]
npc = SimpleDqnNpcV3(observation_size, num_of_actions)
is_done = False
avgs = []

In [ ]:
for model in range(NUM_OF_AGENTS):
    scores = []
    for episode in range(NUM_OF_EPISODES):
        score = 0
        current_state = np.reshape(game.reset(), [1, observation_size])
        for frame in range(FRAMES_PER_EPISODE):
            game.render()
            action = npc.act(current_state)
            new_state, gained_reward, is_done, info = game.step(action)
            new_state = np.reshape(new_state, [1, observation_size])
            npc.retain(current_state, action, gained_reward, new_state, is_done)
            score += gained_reward
            current_state = new_state
            if len(npc.memory) > BATCH_SIZE:
                npc.replay(BATCH_SIZE)
            if is_done:
                break
        scores.append(score)
        npc.save("simple_dqn_" + str(model) + ".h5")
    avgs.append(sum(scores) / len(scores))

In [ ]:
for i, avg in enumerate(avgs):
    print("Model {} has avarage: {}".format(i, avg))
print("Overall avg: {}".format(sum(avgs) / len(avgs)))

Model 0 has avarage: -120.22066950112930
Model 1 has avarage: -117.20091066801283
Model 2 has avarage: -138.07912080607048
Model 3 has avarage: -129.04182009469854
Overall avg: --126.13563026747778



### Wnioski
<p style='text-align: justify;'>
Zgodnie z przewidywaniami agent tym razem nie został przetrenowany. Co więcej, zdobył najlepsze dotychczas rezultaty, 
a nagrane fragmenty rozgrywki tylko to potwierdzają. Poniżej można zobaczyć poszczególne kamienie milowe osiągnięte 
przez ten model:
</p>

1. Jak zwykle niemrawe początki:

<center><img src="videos\v3_0.gif" width="460" height="280" /></center>


2. Po około 20 grach agent podjął pierwsze próby lądowania. Co ciekawe - odkrył, że za używanie głównego silnika 
czeka go kara, więc praktycznie go nie używał:

<center><img src="videos\v3_1.gif" width="460" height="280" /></center>


3. Po rozegraniu około 80 gier agent był w stanie lądować, ale wciąż miał problemy z trafieniem w lądowisko:

<center><img src="videos\v3_2.gif" width="460" height="280" /></center>


4. Po 120 grach poprawiła się celność agenta. Tym razem był w stanie wylądować, ale nie wiedział, że musi wyłączyć silniki,
żeby zakończyć epizod, więc bezwładnie pchał się w bok:

<center><img src="videos\v3_3.gif" width="460" height="280" /></center>


5. Po blisko 160 grach agent nareszcie odkrył, że aby wygrać, musi wyłączyć silnik i tym samym zacząć zdobywać
dodatnią sumę punktów. W poniższej próbie był 5 punktów na plusie:


<center><img src="videos\v3_4.gif" width="460" height="280" /></center>


6. Finalnie model nauczył się lądować i w ciągu ostatnich 20 gier wylądował aż trzykrotnie zdobywając kolejno
40, 20 oraz 80 punktów na plusie. Poniżej prezentujemy to najlepsze lądowanie:
<center><img src="videos\v3_5.gif" width="460" height="280" /></center>


### Posłowie
<p style='text-align: justify;'>
Naszym planem na dalsze usprawnienie sieci neuronowej było przeanalizowanie wag wszystkich połączeń 
aby wykryć te, których wartości są bliskie zera. Jendakże zadanie to byłoby niezwykle 
czasochłonne, więc zdecydowaliśmy się zakończyć prace nad siecią na tym etapie, 
aby przeprowadzić eksperyment z algorytmem genetycznym.
</p>

